In [51]:
import psycopg2
import pandas as pd
import geopandas as gpd
import numpy as np
import json
from sqlalchemy import create_engine, text

In [4]:
# Define database connection parameters
database_connection = {
    'drivername': 'postgresql',
    'username': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432',
    'database': 'vic_db',
}


# A psycopg2 connection and cursor
conn = psycopg2.connect(user=database_connection['username'],
                        password=database_connection['password'],
                        host=database_connection['host'],
                        port=database_connection['port'],
                        database=database_connection['database'])
conn.autocommit = True
cursor = conn.cursor()

# Create a SQLAlchemy engine
engine = create_engine('postgresql://%(username)s:%(password)s@%(host)s/%(database)s' % database_connection, isolation_level="AUTOCOMMIT")
conn_alchemy = engine.connect()

In [41]:

# Define the SQL query to select data from the PostGIS table
sql = "SELECT * FROM vmtrans.tr_road_all LIMIT 10;"

# Read data from PostGIS into a GeoDataFrame
gdf = gpd.read_postgis(sql, con=engine)

# # Print the GeoDataFrame
# print(gdf)


In [9]:
# List all tables
cursor.execute("SELECT table_schema, table_name FROM information_schema.tables;")
result_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

In [11]:
cursor.execute("SELECT * FROM vmtrans.tr_road_all LIMIT 10;")
result_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

In [37]:
# In table tr_road_all, create a column called "road_length" which is the length of the road segment
# But not really create it, just create a new column from SELECT statement. Use meters as unit, so we have to convert the length from degree to meters
# cursor.execute("SELECT SUM(ST_Length(geom::geography)) as road_length FROM vmtrans.tr_road_all WHERE ezi_road_name LIKE '%GLENBROOK AVENUE%';")
cursor.execute(
"""
SELECT 
    ST_Length(geom::geography) as road_length,
    ST_AsText(ST_StartPoint(ST_GeometryN(geom, 1))) AS first_point,
    ST_AsText(ST_EndPoint(ST_GeometryN(geom, 1))) AS last_point
FROM vmtrans.tr_road_all 
WHERE ezi_road_name LIKE '%GLENBROOK AVENUE%';
""")
result_df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])
result_df

,road_length,first_point,last_point
0,74.321053,POINT(144.7845466330001 -37.765636496999946),POINT(144.78466143100002 -37.76497362999993)
1,114.023511,POINT(145.0549611360001 -37.86946304899993),POINT(145.05515306400002 -37.86844708399997)
2,271.300323,POINT(145.12387210600002 -37.90911573999995),POINT(145.12431451100008 -37.90669674099996)
3,52.212317,POINT(145.0548732430001 -37.86992826699998),POINT(145.0549611360001 -37.86946304899993)
4,247.812288,POINT(144.78573093800003 -37.76742263799997),POINT(144.7845466330001 -37.765636496999946)
5,13.516021,POINT(145.12301246200002 -37.91167085699993),POINT(145.12310854800012 -37.911575813999946)
6,287.039748,POINT(145.12310854800012 -37.911575813999946),POINT(145.12387210600002 -37.90911573999995)
7,17.342869,POINT(145.1290169560001 -38.05468524999998),POINT(145.12883521900005 -38.05474658899993)
8,301.855147,POINT(145.05515306400002 -37.86844708399997),POINT(145.0556611290001 -37.86575751099997)
9,164.729226,POINT(144.7879104740001 -37.76769546799994),POINT(144.7860614210001 -37.76747599199996)


In [59]:
# sql = """
# SELECT ezi_road_name, ST_Length(geom::geography) as road_length, ST_AsText(ST_StartPoint(ST_GeometryN(geom, 1))) AS first_point, ST_AsText(ST_EndPoint(ST_GeometryN(geom, 1))) AS last_point FROM vmtrans.tr_road_all WHERE ezi_road_name LIKE '%GLENBROOK AVENUE%';
# """
sql = """
SELECT
    GeometryType(geom) AS geometry_type,
    COUNT(*) AS count
FROM vmtrans.tr_road_all
GROUP BY GeometryType(geom);
"""

pd.read_sql_query(text(sql), conn_alchemy)

,geometry_type,count
0,MULTILINESTRING,1222415
